<a href="https://colab.research.google.com/github/Remon19/journey-into-ML/blob/main/ResNet_for_cats_%26_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
import numpy as np
import os
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
import torch.cuda as cuda
import time
from torch.autograd import Variable

In [2]:
!git clone https://github.com/ardamavi/Dog-Cat-Classifier.git

Cloning into 'Dog-Cat-Classifier'...
remote: Enumerating objects: 1654, done.
remote: Total 1654 (delta 0), reused 0 (delta 0), pack-reused 1654
Receiving objects: 100% (1654/1654), 34.83 MiB | 43.55 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [3]:
class NeuralNet(nn.Module):
  def __init__(self,input_size, n_nodes, output_size) -> None:
      super(NeuralNet, self).__init__()
      self.layer1 = nn.Linear(input_size, n_nodes)
      self.output_layer = nn.Linear(n_nodes, output_size)

  def forward(self, x):
    x = self.layer1(x)
    x = F.relu(x)
    x = self.output_layer(x)
    x = F.sigmoid(x)
    return x



In [4]:
path = '/content/Dog-Cat-Classifier/Data/Train_Data/'
dog_files = [f for f in glob.glob(path +'dog/*.jpg')]
cat_files = [f for f in glob.glob(path + 'cat/*.jpg')]
files = dog_files + cat_files
print(f'Total no of images {len(files)}')
no_of_images = len(files)

Total no of images 1399


In [5]:
shuffle = np.random.permutation(no_of_images)

In [6]:
os.mkdir(os.path.join(path,'train'))
os.mkdir(os.path.join(path,'valid'))
for t in ['train','valid']:
  for folder in ['dog/','cat/']:
    os.mkdir(os.path.join(path,t,folder))

In [7]:
for i in shuffle[:250]:
  folder = files[i].split('/')[-2].split('.')[0]
  image = files[i].split('/')[-1]
  os.rename(files[i],os.path.join(path,'valid',folder,image))

In [8]:
for i in shuffle[250:]:
  folder = files[i].split('/')[-2].split('.')[0]
  image = files[i].split('/')[-1]
  os.rename(files[i],os.path.join(path,'train',folder,image))

In [9]:
transform = transforms.Compose([transforms.Resize((224,224))
                              ,transforms.ToTensor()
                              ,transforms.Normalize([0.12, 0.11,
0.40], [0.89, 0.21, 0.12])])
train = ImageFolder(os.path.join(path,'train/'),transform)
valid = ImageFolder(os.path.join(path,'valid/'),transform)

In [27]:
train_data_generator = torch.utils.data.DataLoader(train,shuffle=True,batch_size=64,num_workers=8)
valid_data_generator = torch.utils.data.DataLoader(valid,batch_size=64,num_workers=8)
dataset_sizes = {'train': len(train), 'valid': len(valid)}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [39]:
pretrained_resnet = models.resnet18(pretrained=True)
number_features = pretrained_resnet.fc.in_features
pretrained_resnet.fc = nn.Linear(number_features, 4)
if cuda.is_available:
  pretrained_resnet = pretrained_resnet.cuda()

In [40]:
learning_rate = 0.005
criterion = nn.CrossEntropyLoss()
fit_optimizer = torch.optim.SGD(pretrained_resnet.parameters(), lr=0.005,
momentum=0.6)
exp_learning_rate_scheduler = torch.optim.lr_scheduler.StepLR(fit_optimizer,
step_size=2, gamma=0.05)
dataloaders = {'train': train_data_generator, 'valid' : valid_data_generator }

In [41]:
def train_my_model(model, dataloaders,criterion, optimizer, scheduler,number_epochs=20):
  since = time.time()
  best_model_weights = model.state_dict()
  best_accuracy = 0.0
  for epoch in range(number_epochs):
    print('Epoch {}/{}'.format(epoch, number_epochs - 1))
    print('-' * 10)
    for each_phase in ['train', 'valid']:
      if each_phase == 'train':
        scheduler.step()
        model.train(True)
      else:
        model.train(False)
      running_loss = 0.0
      running_corrects = 0
      for data in dataloaders[each_phase]:
        input_data, label_data = data
        if cuda.is_available():
          input_data = Variable(input_data.cuda())
          label_data = Variable(label_data.cuda())
        else:
          input_data, label_data = Variable(input_data), Variable(label_data)
        optimizer.zero_grad()
        outputs = model(input_data)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, label_data)
        if each_phase == 'train':
          loss.backward()
          optimizer.step()
        running_loss += loss.data
        running_corrects += torch.sum(preds == label_data.data)
        epoch_loss = running_loss / dataset_sizes[each_phase]
        epoch_acc = running_corrects / dataset_sizes[each_phase]
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(each_phase,
        epoch_loss, epoch_acc))
        if each_phase == 'valid' and epoch_acc > best_accuracy:
          best_accuracy = epoch_acc
          best_model_weights = model.state_dict()
  print()
  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
  time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_accuracy))
  model.load_state_dict(best_model_weights)
  return model




In [42]:
train_my_model(pretrained_resnet,dataloaders, criterion, fit_optimizer,
exp_learning_rate_scheduler, number_epochs=20)

Epoch 0/19
----------


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_che

train Loss: 0.0013 Acc: 0.0070
train Loss: 0.0021 Acc: 0.0453
train Loss: 0.0027 Acc: 0.0870
train Loss: 0.0032 Acc: 0.1323
train Loss: 0.0036 Acc: 0.1828
train Loss: 0.0040 Acc: 0.2289
train Loss: 0.0043 Acc: 0.2794
train Loss: 0.0045 Acc: 0.3307
train Loss: 0.0048 Acc: 0.3795
train Loss: 0.0050 Acc: 0.4308
train Loss: 0.0052 Acc: 0.4813
train Loss: 0.0054 Acc: 0.5335
train Loss: 0.0056 Acc: 0.5857
train Loss: 0.0058 Acc: 0.6388
train Loss: 0.0059 Acc: 0.6919
train Loss: 0.0061 Acc: 0.7467
train Loss: 0.0062 Acc: 0.8016
train Loss: 0.0063 Acc: 0.8538
valid Loss: 0.0007 Acc: 0.2440
valid Loss: 0.0014 Acc: 0.4840
valid Loss: 0.0020 Acc: 0.7200
valid Loss: 0.0025 Acc: 0.9480
Epoch 1/19
----------
train Loss: 0.0001 Acc: 0.0540
train Loss: 0.0003 Acc: 0.1062
train Loss: 0.0004 Acc: 0.1601
train Loss: 0.0005 Acc: 0.2150
train Loss: 0.0006 Acc: 0.2698
train Loss: 0.0008 Acc: 0.3255
train Loss: 0.0009 Acc: 0.3803
train Loss: 0.0010 Acc: 0.4360
train Loss: 0.0010 Acc: 0.4917
train Loss: 0.001

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  